In [1]:
# Imports

import os

from gerrychain import Graph, GeographicPartition, Partition, Election
from gerrychain.updaters import Tally, cut_edges
import geopandas as gpd
import numpy as np
import random
import copy

from gerrychain import MarkovChain
from gerrychain.constraints import single_flip_contiguous
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
from gerrychain.metrics import polsby_popper
from gerrychain import constraints

import matplotlib.pyplot as plt

import networkx as nx

import pandas

import math

from functools import partial

In [ ]:
class Annealer:
    def __init__(self, acceptor, temp):
        self.counter = 0 # counts number of steps in the chain, increment on accept
        self.acceptor = acceptor #acceptance function, eats a partition, returns a bool
        self.temp = temp # function which eats a count and gives the current temp
        
        
        
    def __call__(self, partition):
        
        if f(partition, counter) is True:
            counter +=1
            return True
        else:
            return False

In [2]:
## Methods

def analyze_dem_seats():

    d_percents = [sorted(partition["SEN12"].percents("Dem")) for partition in chain]
    data = pandas.DataFrame(d_percents)

    ax = data.boxplot(positions=range(len(data.columns)))
    data.iloc[0].plot(style="ro", ax=ax)

    plt.show()

def deviation(values):
    ideal = np.mean(values)
    deviations = [ np.abs(x - ideal)/ideal for x in values]
    return np.max(deviations)

def MattinglyScore_2014(partition, L = 1/2, beta = 1):
    #L is Mattingly's Lambda

    #Based on this one: https://arxiv.org/pdf/1410.8796.pdf
    #This was the original project, and was a prototype

    c_pop = 1/5000
    c_compact = 2000

    J_pop = c_pop*np.var(list(partition["population"].values()))
    J_compact = c_compact * np.sum( list( partition["polsby_popper"]))

    J_L = L * J_pop + (1 - L) * J_compact


    return np.exp(e, -1 * beta * J_L)

def MattinglyScore_2018(partition, df):
    # Based on this one https: // arxiv.org / pdf / 1801.03783.pdf
    # This is the most polished version available


    M_C = 10000


    ###Population Part

    pop_list = list(partition["population"].values())
    pop_ideal = np.sum ( pop_list) / 13
    J_p = np.linalg.norm(( [ x/ pop_ideal - 1 for x in pop_list]))


    ###PolsbyPopperPart

    J_I = np.sum( list( partition["polsby_popper"]))

    ###County Splits

    df["current"] = df[unique_label].map(partition.assignment)
    county_splits = {} #Maps to list of number of VTDS in each districts

    for pair in df.groupby(["COUNTYFP10", "current"])["GEOID10"]:
        county_splits[pair[0][0]] = []

    for pair in df.groupby(["COUNTYFP10", "current"])["GEOID10"]:
        county_splits[pair[0][0]].append(len(pair[1]))

    # Checking against num splits: len ( [x for x in county_splits.keys() if len(county_splits[x]) > 1] )

    num_2_splits = 0
    num_2_splits_W = 0
    num_greater_splits = 0
    num_greater_splits_W = 0


    for county in county_splits.keys():
        if len( county_splits[county]) == 2:
            total = sum( county_splits[county])
            max_2 = min( county_splits[county])
            num_2_splits += 1
            num_2_splits_W += np.sqrt( max_2 / total )
        if len(county_splits[county]) > 2:
            total = sum(county_splits[county])
            county_splits[county].sort()
            left_overs = total - county_splits[county][-1] - county_splits[county][-2]
            num_greater_splits += 1
            num_greater_splits_W += np.sqrt( left_overs / total)

    J_c = num_2_splits * num_2_splits_W + M_C * num_greater_splits * num_greater_splits_W

    ##VRA score --> @Daryl
        
    # mattingly's "poor man's vra" is sqrt(H(.4448 - m_1)) + sqrt(H(.3620 - m_2)) where
    # m_1 and m_2 are the BPOP for the two districts with the highest BPOP
    # and H(x) = max(0,x) (the name H comes from 'hinge', bc this looks like the hinge loss in ML)
    
    
    pop_list = list(partition["population"].values())
    bpop_list = list(partition["black_pop"].values())
    
    bpop_props = sorted([ bpop_list[i]/pop_list[i] for i in range(len(pop_list)) ], reverse=True)
    
    J_m = math.sqrt( math.max(0,(.4448 - bpop_props[0]))   ) + math.sqrt(  math.max(.3620 - bpop_props[1])  )

    return score

def num_splits(partition, df):
    df["current"] = df.index.map(dict(partition.assignment))
    return sum(df.groupby("COUNTYFP10")["current"].nunique() > 1)

def pop_MCMC(partition):
    temperature = 1000
    bound = 1
    if partition.parent is not None:
        parent_score = np.var(list(partition.parent["population"].values()))
        current_score = np.var(list(partition["population"].values()))
        if parent_score > current_score:
            bound = 1
        else:
            bound = (parent_score / current_score)**temperature
            #print('bound is:', bound)
    return random.random() < bound

def polsby_MCMC(partition):
    temperature = 100000
    bound = 1
    if partition.parent is not None:
        parent_score =  1 - np.mean(list(partition.parent["polsby_popper"].values()))
        current_score = 1 - np.mean(list(partition["polsby_popper"].values()))
        if parent_score > current_score:
            bound = 1
        else:
            bound = (parent_score / current_score)**(temperature)
            #print('bound is:', bound)
    return random.random() < bound


def popandpolsby_MCMC(partition):
    return polsby_MCMC(partition) and pop_MCMC(partition)





def mattingly_accept(partition, temperature):
    pass


gc_mattingly_accept = partial()

In [5]:
# setup -- SLOW

shapefile = "NC_VTD/NC_VTD.shp"

df = gpd.read_file(shapefile)



for idx, row in df.iterrows():
    try:
        row.geometry.intersection(row.geometry)
    except TopologicalError:
        buffered = row.geometry.buffer(0)
        buffered.intersection(buffered)
        repaired.append(idx)
        row.geometry = buffered

        
graph = Graph.from_geodataframe(df,ignore_errors=True)        
  

In [6]:
graph.add_data(df,list(df))
print(list(df))

['ALAND10', 'AWATER10', 'VTD', 'County', 'VTD_Key', 'VTD_Name', 'PL10AA_TOT', 'PL10VA_TOT', 'EL08G_GV_D', 'EL08G_GV_R', 'EL08G_GV_L', 'EL08G_GV_T', 'EL08G_USS_', 'EL08G_US_1', 'EL08G_US_2', 'EL08G_US_3', 'EL08G_US_4', 'EL10G_USS_', 'EL10G_US_1', 'EL10G_US_2', 'EL10G_US_3', 'EL10G_US_4', 'EL12G_GV_D', 'EL12G_GV_R', 'EL12G_GV_L', 'EL12G_GV_W', 'EL12G_GV_1', 'EL12G_GV_T', 'EL14G_USS_', 'EL14G_US_1', 'EL14G_US_2', 'EL14G_US_3', 'EL14G_US_4', 'Shape_Leng', 'Shape_Area', 'EL12G_PR_D', 'EL12G_PR_R', 'EL12G_PR_L', 'EL12G_PR_W', 'EL12G_PR_1', 'EL12G_PR_T', 'EL16G_PR_R', 'EL16G_PR_D', 'EL16G_PR_L', 'EL16G_PR_W', 'EL16G_PR_T', 'EL16G_USS_', 'EL16G_US_1', 'EL16G_US_2', 'EL16G_US_3', 'EL16G_GV_D', 'EL16G_GV_R', 'EL16G_GV_L', 'EL16G_GV_T', 'BPOP', 'nBPOP', 'judge', 'newplan', 'oldplan', 'TOTPOP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'HISP', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP',

In [8]:
election = Election("SEN12", {"Dem": "EL14G_USS_", "Rep": "EL14G_US_1"})

starting_partition = GeographicPartition(
    graph,
    assignment="judge",
    updaters={
        "polsby_popper" : polsby_popper,
        "cut_edges": cut_edges,
        "population": Tally("PL10AA_TOT", alias="population"),
        "black_pop": Tally("BPOP", alias="black_pop"), #this is black pop, for BVAP, use 'BVAP' (nb: BVAP doesn't include black hispanic but BPOP does)
        "SEN12": election,
        "County Splits": num_splits
    }
)

print(single_flip_contiguous(starting_partition))

True


In [40]:
#run with MCMC constraints

#This is a hack : because of somethign in continuiguity check
#starting_partition.parent = starting_partition
#End hack

chain = MarkovChain(
    proposal=propose_random_flip,
    constraints=[single_flip_contiguous],
    accept=popandpolsby_MCMC,
    initial_state=starting_partition,
    total_steps=10000
)

for part in chain:
    pass
    #print(deviation(list(part["population"].values())))


KeyboardInterrupt: 

In [ ]:
# run without constraints

chain = MarkovChain(
    proposal=propose_random_flip,
    constraints=[single_flip_contiguous],
    accept=always_accept,
    initial_state=starting_partition,
    total_steps=10000
)
for part in chain:
    pass

In [ ]:
# mattingly 2018 annealing

chain = MarkovChain(
    proposal=propose_random_flip,
    constraints=[single_flip_contiguous],
    accept=always_accept,
    initial_state=starting_partition,
    total_steps=40000
)
for part in chain:
    pass

## Use an updated to  grow beta linearly over 60,000 accepted steps
## -> @Daryl

## beta fixed at one for 20000 *accepted* steps

#At the end of this, we get one sample

In [ ]:
# test annealing

stages = 5

annealed_partitions = []

for k in range(stages):
    starting_partition = run_without_constraints(starting_partition)
    annealed_partition = run_with_MCMC_constraints(starting_partition)
    print( "unnealed", [deviation(list(starting_partition["population"].values())), np.mean(list(starting_partition["polsby_popper"].values()))] )
    print(  "annealed", [deviation(list(annealed_partition["population"].values())), np.mean(list(annealed_partition["polsby_popper"].values()))] )
    annealed_partitions.append(annealed_partition)
    starting_partition = annealed_partition
